In [1]:
import os, sys
import ase
import ase.calculators.vasp
import ase.io 
import ase.units
from ase.visualize import view
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import mywrapper

# When we are mixing two or more species, VASP manual suggests that its typically a good idea to use `1.3*max(ENMAX)` of all involved species



In [2]:
!grep "ENMAX" $VASP_PP_PATH/potpaw_PBE/Ti/POTCAR
!grep "ENMAX" $VASP_PP_PATH/potpaw_PBE/O/POTCAR

   ENMAX  =  178.330; ENMIN  =  133.747 eV
   ENMAX  =  400.000; ENMIN  =  300.000 eV


In [3]:
atoms = ase.io.read('TiO2_conventional.cif')
natom = len(atoms.get_positions())
view(atoms)
cell = atoms.get_cell()
ase.visualize.view(atoms)
print(cell)

[[3.80613435 0.         0.        ]
 [0.         3.80613435 0.        ]
 [0.         0.         9.78435544]]


## defining k grid as it is not cubic

In [6]:
alatmag_kpt = 30    # why 30 we know na (a1*k1 ~ 30)
amag = []
for i in range(3):
    amag.append(np.sqrt(np.sum(cell[i]**2)))  # 
print(amag)    
kgrid = [int(round(alatmag_kpt/amag[0])), int(round(alatmag_kpt/amag[1])), int(round(alatmag_kpt/amag[2]))]
print(kgrid)


[3.80613435, 3.80613435, 9.78435544]
[8, 8, 3]


In [7]:
for alatmag_kpt in [24,30,36,42]:
    kgrid = [int(round(alatmag_kpt/amag[0])), int(round(alatmag_kpt/amag[1])), int(round(alatmag_kpt/amag[2]))]
    print(kgrid)

[6, 6, 2]
[8, 8, 3]
[9, 9, 4]
[11, 11, 4]


In [9]:
# relax directly with isif=3
path = "TiO2/one_step_relax"

calc = ase.calculators.vasp.Vasp(
            xc="pbe",
            kpts=kgrid,   # typically we prefer equal sampling [kpts*cell_length] in each direction
            encut=520,
            ediff=1e-5,
            algo='fast',
            prec='accurate',
            nelmin=4,
            nelm=100,
            ismear=0,
            sigma=0.05,
            lwave=False,   
            npar=2, 
            isif=3,
            nsw=100,
            ediffg=1e-4,
            ibrion=2
            )   
atoms.set_calculator(calc)
energy_oneStep = mywrapper.get_energy(atoms, dir=path)
print('energy after relaxation with ISIF-2: %5.3f' % (energy_oneStep))

energy after relaxation with ISIF-2: -105.958


Now lets do a multi-step relaxation with: <br/>
- Step-1: ISIF=2
- Step-2: ISIF=4
- Step-3: ISIF=3

In [10]:
# step-1
step1_done = False
step2_done = False
atoms = ase.io.read('TiO2_conventional.cif')

path = "TiO2/multistep/step1"
calc = ase.calculators.vasp.Vasp(
            xc="pbe",
            kpts=[8,8,4],   # typically we prefer equal sampling [kpts*cell_length] in each direction
            encut=520,
            ediff=1e-5,
            algo='fast',
            prec='accurate',
            nelmin=4,
            nelm=100,
            ismear=0,
            sigma=0.05,
            lwave=False,   
            npar=2, 
            isif=2,
            nsw=100,
            ediffg=natom*(1e-3),
            ibrion=2
            )   

atoms.set_calculator(calc)
energy_step1 = mywrapper.get_energy(atoms, dir=path)
calc = mywrapper.get_calculator(dir=path)

if calc is not None:
    step1_done = True
    atoms = calc.atoms      # vasp writes latest relaxed coordinate sin the CONTCAR file

print('energy after relaxation with ISIF-2: %5.3f' % (energy_step1))

energy after relaxation with ISIF-2: -105.955


In [11]:
# step-2

if step1_done:
    path = "TiO2/multistep/step2"
    calc = ase.calculators.vasp.Vasp(
            xc="pbe",
            kpts=kgrid,   # typically we prefer equal sampling [kpts*cell_length] in each direction
            encut=520,
            ediff=1e-5,
            algo='fast',
            prec='accurate',
            nelmin=4,
            nelm=100,
            ismear=0,
            sigma=0.05,
            lwave=False,   
            npar=2, 
            isif=4,
            nsw=100,
            ediffg=natom*(1e-3),
            ibrion=2
            )   

    atoms.set_calculator(calc)
    energy_step2 = mywrapper.get_energy(atoms, dir=path)
    calc = mywrapper.get_calculator(dir=path)

    if calc is not None:
        step2_done = True
        atoms = calc.atoms     # vasp writes latest relaxed coordinate sin the CONTCAR file
    
    print('energy after relaxation with ISIF-4: %5.3f' % (energy_step2))

energy after relaxation with ISIF-4: -105.958


In [12]:
# do the third and final step
# step-3

if step2_done:
    path = "TiO2/multistep/step3"
    calc = ase.calculators.vasp.Vasp(
            xc="pbe",
            kpts=kgrid,   # typically we prefer equal sampling [kpts*cell_length] in each direction
            encut=520,
            ediff=1e-5,
            algo='fast',
            prec='accurate',
            nelmin=4,
            nelm=100,
            ismear=0,
            sigma=0.05,
            lwave=False,   
            npar=2, 
            isif=3,
            nsw=100,
            ediffg=natom*(1e-3),
            ibrion=2
            )   

    atoms.set_calculator(calc)
    energy_multiStep = mywrapper.get_energy(atoms, dir=path)
    calc = mywrapper.get_calculator(dir=path)

    if calc is not None:
        atoms = calc.atoms      # vasp writes latest relaxed coordinate sin the CONTCAR file
        
    
    # let us know look at the energy difference
    print('\nSummary:')
    print("Energy after single-step relaxation: %5.3f " % (energy_oneStep))
    print("Energy after multi-step step-1: %5.3f " % (energy_step1))
    print("Energy after multi-step step-2: %5.3f " % (energy_step2))
    print("Energy after multi-step step-3: %5.3f " % (energy_multiStep))


Summary:
Energy after single-step relaxation: -105.958 
Energy after multi-step step-1: -105.955 
Energy after multi-step step-2: -105.958 
Energy after multi-step step-3: -105.960 
